<a href="https://colab.research.google.com/github/ad5454/Fake_news_classifier/blob/main/Fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Loading Data From Kaggle

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle competitions download -c fake-news

 77% 36.0M/46.5M [00:00<00:00, 182MB/s] 
100% 46.5M/46.5M [00:00<00:00, 168MB/s]


In [10]:
!unzip /content/sample_data/california_housing_train.csv
!unzip /content/fake-news.zip

Archive:  /content/sample_data/california_housing_train.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/sample_data/california_housing_train.csv or
        /content/sample_data/california_housing_train.csv.zip, and cannot find /content/sample_data/california_housing_train.csv.ZIP, period.
Archive:  /content/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


###Importing Data

In [11]:
import pandas as pd
data=pd.read_csv("train.csv")
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


###Removing NaN Values from the dataset

In [12]:
data=data.dropna()
data.reset_index(inplace=True)

###Extracting Dependent and Independent variables from data

In [13]:
X = data.drop("label",axis=1)
y = data["label"]

In [14]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


###Importing the required Libraries

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

###Preprocessing The Text



In [16]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
messages=X.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]"," ",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]  # (word not in stopwords) and stem(word)
  review = ' '.join(review)
  corpus.append(review)

### One Hot Encoding

In [18]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus]


In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2772 1960 2165]
 [   0    0    0 ...  584 3205  447]
 [   0    0    0 ... 1101  275 4699]
 ...
 [   0    0    0 ... 2992 4429 3350]
 [   0    0    0 ... 2260   21 2216]
 [   0    0    0 ...  793 3398 4320]]


#Creating Model for Training

In [20]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [23]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 8s 33ms/step - loss: 0.3316 - accuracy: 0.8471 - val_loss: 0.1930 - val_accuracy: 0.9140
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1365 - accuracy: 0.9469 - val_loss: 0.1916 - val_accuracy: 0.9193
Epoch 3/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0956 - accuracy: 0.9642 - val_loss: 0.2165 - val_accuracy: 0.9191
Epoch 4/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0691 - accuracy: 0.9760 - val_loss: 0.2292 - val_accuracy: 0.9178
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0514 - accuracy: 0.9828 - val_loss: 0.2661 - val_accuracy: 0.9142
Epoch 6/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0298 - accuracy: 0.9900 - val_loss: 0.3477 - val_accuracy: 0.9128
Epoch 7/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0197 - accuracy: 0.9942 - val_loss: 0.4579 - val_accuracy: 0.9120

###Performance Metrics And Accuracy



In [24]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

189/189 [==============================] - 1s 4ms/step


array([[3148,  271],
       [ 272, 2344]])

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.9100248550124275